Single Precision

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from jax import config
config.update("jax_enable_x64", False)
config.update("jax_debug_nans", True)

In [ ]:
import OTF, DatasetGenerator
import numpy as np
import pandas as pd
import optax
import matplotlib.pyplot as plt

from jax import jit, grad, value_and_grad, random
from flax.training import checkpoints

# plotting, move this to seperate notebook
from jax import vmap
import jax.numpy as jnp
from matplotlib import colormaps

In [ ]:
@jit
def step(opt_state, params, batch, solver_steps):
    # from optax doc: https://optax.readthedocs.io/en/latest/gradient_accumulation.html
    loss, grads = value_and_grad(LOSS_FUNC)(params, batch, **LOSS_KWARGS, solver_steps=solver_steps)
    updates, opt_state = gradient_transform.update(grads, opt_state, params=params) 
    params = optax.apply_updates(params, updates)
    return opt_state, params, loss

In [ ]:
def eval_val(val_data, params, solver_steps):
    return VAL_LOSS_FUNC(params=params, batch=val_data, **VAL_LOSS_KWARGS, solver_steps=solver_steps)

In [ ]:
def train_step(train_data, opt_state, params, solver_steps):
    opt_state, params, loss = step(opt_state, params, batch=train_data, solver_steps=solver_steps)
    return opt_state, params, loss

In [ ]:
MODEL_PREFIX = 'OFT'
PRECISION_PREFIX = 'float32'

In [ ]:
SOLVER_STEPS = 20
EVAL_SOLVER_STEPS = 20

In [ ]:
def run(train_re, params, opt_state, data_key):
    train_losses  = []
    val_losses, val_inv_errors, val_mmds = [], [], []
    best_val_mmd, best_epoch_mmd, best_params_mmd = np.inf, None, None
    best_val_loss, best_epoch_loss, best_params_loss = np.inf, None, None
    
    if report_as_csv:
        report_df = pd.DataFrame({'Epoch': [], 
                                  'Training Loss': [], 
                                  'Validation Loss': [], 
                                  'Validation Inverse Error': [],
                                  'Validation MMD': [],
                                  'Best Epoch Loss': [], 
                                  'Best Epoch MMD': [], 
                                  'Best Validation Loss': [],
                                  'Best Validation MMD': [],})

    for epoch in range(1, EPOCHS + 1):
        opt_state, params, train_loss = train_step(train_re, opt_state, params, SOLVER_STEPS)    
        train_losses.append(train_loss)

        val_loss, val_inv_error, val_mmd  = eval_val(val, params, EVAL_SOLVER_STEPS)
        print(f'Epoch {epoch} --- Train Loss: {np.mean(train_loss)}, Val Loss: {val_loss}, Val Inv Error: {val_inv_error}, Val MMD: {val_mmd}')
        val_losses.append(val_loss)
        val_inv_errors.append(val_inv_error)
        val_mmds.append(val_mmd)

        if epoch >= 5:
            val_loss_rolling = np.mean(val_losses[epoch-5:])    
            if val_loss_rolling < best_val_loss:
                best_epoch_loss, best_params_loss, best_val_loss = epoch, params, val_loss
                
            val_mmd_rolling = np.mean(val_mmds[epoch-5:])    
            if val_mmd_rolling < best_val_mmd:
                best_epoch_mmd, best_params_mmd, best_val_mmd = epoch, params, val_mmd        
        

        # resample training data
        if epoch % 25 == 0:    
            data_key = random.fold_in(data_key, epoch)
            train_re = TRAIN_GENERATOR(data_key, training_size, dtype=PRECISION_PREFIX, **TRAIN_GEN_KWARGS)      

        if report_as_csv:    
            report_df.loc[epoch - 1] = [epoch, train_loss,
                                        val_loss, val_inv_error, val_mmd, 
                                        best_epoch_loss, best_epoch_mmd, best_val_loss, best_val_mmd]
            if epoch % 10 == 0: 
                report_df.to_csv(report_filename)


    print(f'\nBest Epoch MMD: {best_epoch_mmd} --- Val MMD: {best_val_mmd}')
    test_loss, test_inv_error, test_mmd = eval_val(test, best_params_mmd, EVAL_SOLVER_STEPS)
    print(f'Best Model MMD --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')
    
    print(f'\nBest Epoch Loss: {best_epoch_loss} --- Val Loss: {best_val_loss}')
    test_loss, test_inv_error, test_mmd = eval_val(test, best_params_loss, EVAL_SOLVER_STEPS)
    print(f'Best Model Loss --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')
    
    ret = (train_losses, val_losses, val_mmds, val_inv_errors, 
           best_epoch_mmd, best_val_mmd, best_params_mmd, 
           best_epoch_loss, best_val_loss, best_params_loss, params)

    return ret

# TUC Letters (no Spacing)

In [ ]:
key = random.PRNGKey(seed=42)

In [ ]:
training_size = 10000
validation_size = 10000
test_size = 20000
spacing = 0

In [ ]:
BATCH_SIZE_TR = training_size
BATCH_SIZE_VAL_TE = validation_size

In [ ]:
train, val, test = DatasetGenerator.make_tuc_letters(
    key, 
    training_size, 
    validation_size, 
    test_size, 
    dtype=PRECISION_PREFIX,
    spacing=spacing
)
normal_sample = random.normal(random.fold_in(key, 42), shape=(validation_size, 2), dtype=PRECISION_PREFIX)

In [ ]:
DATASET_PREFIX = 'letters-nospacing'

In [ ]:
TRAIN_GENERATOR = DatasetGenerator.make_tuc_letters_tr
TRAIN_GEN_KWARGS = {'spacing': spacing}

## 5 Potential

In [ ]:
NPOT_PREFIX = '5pot'

In [ ]:
key = random.PRNGKey(42)
data_key, key = random.split(key, 2)

In [ ]:
EPOCHS = 20000
MODEL = OTF.OTF(
        input_dim=2, 
        hidden_dim=32,
        resnet_depth=2,
        rank=10,
        key=key, 
        phi=OTF.Phi,
        alpha1=15.,
        alpha2=2.,
        num_blocks=5,
        t0=0.,
        t1=1.,
        num_steps=20
)
LOSS_FUNC = jit(MODEL._loss)
VAL_LOSS_FUNC = jit(MODEL.metrics)
LOSS_KWARGS = {}
VAL_LOSS_KWARGS = {'normal_batch': normal_sample}

In [ ]:
# Exponential decay of the learning rate.
scheduler = optax.exponential_decay(
    init_value=5e-2, 
    transition_steps=1000,
    transition_begin=0,
    decay_rate=0.6)

gradient_transform = optax.chain(
    optax.scale_by_adam(),              # Use the updates from adam.
    optax.scale_by_schedule(scheduler), # Adapt LR
    optax.scale(-1.0)                   # Scale updates by -1 since optax.apply_updates 
                                        # is additive and we want to descend on the loss.
)

In [ ]:
params = MODEL.params
opt_state = gradient_transform.init(params)

In [ ]:
report_as_csv = False
report_filename = filestr = '_'.join([MODEL_PREFIX, PRECISION_PREFIX, DATASET_PREFIX, NPOT_PREFIX]) + '.csv'

In [ ]:
train_re, params, opt_state, data_key = train, params, opt_state, data_key

In [ ]:
train_losses  = []
val_losses, val_inv_errors, val_mmds = [], [], []
best_val_mmd, best_epoch_mmd, best_params_mmd = np.inf, None, None
best_val_loss, best_epoch_loss, best_params_loss = np.inf, None, None

if report_as_csv:
    report_df = pd.DataFrame({'Epoch': [], 
                              'Training Loss': [], 
                              'Validation Loss': [], 
                              'Validation Inverse Error': [],
                              'Validation MMD': [],
                              'Best Epoch Loss': [], 
                              'Best Epoch MMD': [], 
                              'Best Validation Loss': [],
                              'Best Validation MMD': [],})

for epoch in range(1, EPOCHS + 1):
    opt_state, params, train_loss = train_step(train_re, opt_state, params, SOLVER_STEPS)    
    train_losses.append(train_loss)

    val_loss, val_inv_error, val_mmd  = eval_val(val, params, EVAL_SOLVER_STEPS)
    print(f'Epoch {epoch} --- Train Loss: {np.mean(train_loss)}, Val Loss: {val_loss}, Val Inv Error: {val_inv_error}, Val MMD: {val_mmd}')
    val_losses.append(val_loss)
    val_inv_errors.append(val_inv_error)
    val_mmds.append(val_mmd)

    if epoch >= 5:
        val_loss_rolling = np.mean(val_losses[epoch-5:])    
        if val_loss_rolling < best_val_loss:
            best_epoch_loss, best_params_loss, best_val_loss = epoch, params, val_loss

        val_mmd_rolling = np.mean(val_mmds[epoch-5:])    
        if val_mmd_rolling < best_val_mmd:
            best_epoch_mmd, best_params_mmd, best_val_mmd = epoch, params, val_mmd        


    # resample training data
    if epoch % 25 == 0:    
        data_key = random.fold_in(data_key, epoch)
        train_re = TRAIN_GENERATOR(data_key, training_size, dtype=PRECISION_PREFIX, **TRAIN_GEN_KWARGS)      

    if report_as_csv:    
        report_df.loc[epoch - 1] = [epoch, train_loss,
                                    val_loss, val_inv_error, val_mmd, 
                                    best_epoch_loss, best_epoch_mmd, best_val_loss, best_val_mmd]
        if epoch % 10 == 0: 
            report_df.to_csv(report_filename)


print(f'\nBest Epoch MMD: {best_epoch_mmd} --- Val MMD: {best_val_mmd}')
test_loss, test_inv_error, test_mmd = eval_val(test, best_params_mmd, EVAL_SOLVER_STEPS)
print(f'Best Model MMD --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')

print(f'\nBest Epoch Loss: {best_epoch_loss} --- Val Loss: {best_val_loss}')
test_loss, test_inv_error, test_mmd = eval_val(test, best_params_loss, EVAL_SOLVER_STEPS)
print(f'Best Model Loss --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')

In [ ]:
CLR = vmap(MODEL.log_pdf_and_preimage,  (0, None, None, None), 0)(train_re, 20, params, False)

In [ ]:
CLR[0].min()

In [ ]:
CLR[0].max()

In [ ]:
CLR = vmap(MODEL.log_pdf_and_preimage,  (0, None, None, None), 0)(train_re, 100, params, False)

In [ ]:
CLR[0].min()

In [ ]:
CLR[0].max()

In [ ]:
CLR[0].argmax()

In [ ]:
plt.hist(CLR[0], bins=50)

In [ ]:
loss = LOSS_FUNC(params, train_re, **LOSS_KWARGS, solver_steps=20)

In [ ]:
train_re.shape

In [ ]:
7500 + 1250

In [ ]:
train_re[9215:9216]

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9215:9216], **LOSS_KWARGS, solver_steps=20)

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9216:9217, ], **LOSS_KWARGS, solver_steps=20)

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9217:9218], **LOSS_KWARGS, solver_steps=20)

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9218:9219], **LOSS_KWARGS, solver_steps=20)

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9219:9220], **LOSS_KWARGS, solver_steps=20)

In [ ]:
train_re[9219:9220]

In [ ]:
LOSS_FUNC(params, train_re[9219:9220], **LOSS_KWARGS, solver_steps=20)

In [ ]:
grads = grad(MODEL._loss)(params, train_re, **LOSS_KWARGS, solver_steps=40) 

In [ ]:
plt.scatter(*train_re.T, s=0.5)
plt.scatter(*train_re[9219:9220].T, c='red', s=10.)
plt.xlabel(r"$x_1$ component")
plt.ylabel(r"$x_2$ component")
plt.savefig('precision_scatter.png', bbox_inches='tight')

In [ ]:
epoch

In [ ]:
config.update("jax_enable_x64", True)

In [ ]:
grads = grad(LOSS_FUNC)(params, train_re[9219:9220].astype(jnp.float64), **LOSS_KWARGS, solver_steps=20)

In [ ]:
from jax import tree_map

In [ ]:
tree_map(lambda x: x.max(), grads)

In [ ]:
print(f'\nBest Epoch MMD: {best_epoch_mmd} --- Val MMD: {best_val_mmd}')
test_loss, test_inv_error, test_mmd = eval_val(test, best_params_mmd, EVAL_SOLVER_STEPS)
print(f'Best Model MMD --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')

print(f'\nBest Epoch Loss: {best_epoch_loss} --- Val Loss: {best_val_loss}')
test_loss, test_inv_error, test_mmd = eval_val(test, best_params_loss, EVAL_SOLVER_STEPS)
print(f'Best Model Loss --- Test Loss: {test_loss}, Test Inv Error: {test_inv_error}, Test MMD: {test_mmd}')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,6))
ax[0].semilogy(train_losses, label='train loss')
ax[0].semilogy(val_losses, label='validation loss')
ax[1].semilogy(train_losses[1000:], label='train loss')
ax[1].semilogy(val_losses[1000:], label='validation loss')
plt.legend()